Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [1]:
from sqlalchemy import *
import datetime

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [8]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('count', Integer),
)
metadata.create_all(engine) # and create the tables in the database

2016-12-18 21:28:56,888 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2016-12-18 21:28:56,889 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2016-12-18 21:28:56,890 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	timestamp DATETIME, 
	count INTEGER, 
	PRIMARY KEY (id)
)




2016-12-18 21:28:56,898 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2016-12-18 21:28:56,899 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [9]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, count) VALUES (:id, :keywords, :timestamp, :count)'

In [ ]:
insert_stmt = queries.insert().values(keywords="puppies")
str(insert_stmt)

In [ ]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

In [ ]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

## HW6

In [10]:
import mediacloud
mc = mediacloud.api.MediaCloud('c8089139c012edf7dd14661df58f0ec2db439ad1cc8410f74ae84dd32fe2d9b3')

In [13]:
keyword1 = 'clinton'
results1 = mc.sentenceCount(keyword1,
        solr_filter=[mc.publish_date_query( datetime.date(2016, 9, 1), datetime.date(2016, 10, 1) ),
                     'tags_id_media:1' ] )

In [19]:
insert_stmt1 = queries.insert().values(keywords=keyword1, count=results1['count'] )
str(insert_stmt1)
insert_stmt1.compile().params

{'count': 140668, 'keywords': 'clinton', 'timestamp': None}

In [17]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt1)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-18 21:34:14,950 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp, count) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO queries (keywords, timestamp, count) VALUES (?, ?, ?)


2016-12-18 21:34:14,952 INFO sqlalchemy.engine.base.Engine ('clinton', '2016-12-18 21:34:14.950543', 140668)


INFO:sqlalchemy.engine.base.Engine:('clinton', '2016-12-18 21:34:14.950543', 140668)


2016-12-18 21:34:14,955 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


[1]

In [18]:
keyword2 = 'obama'
results2 = mc.sentenceCount(keyword2,
        solr_filter=[mc.publish_date_query( datetime.date(2016, 9, 1), datetime.date(2016, 10, 1) ),
                     'tags_id_media:1' ] )

In [21]:
insert_stmt2 = queries.insert().values(keywords=keyword2, count=results2['count'] )
str(insert_stmt2)

'INSERT INTO queries (keywords, timestamp, count) VALUES (:keywords, :timestamp, :count)'

In [22]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt2)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-18 21:35:55,952 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp, count) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO queries (keywords, timestamp, count) VALUES (?, ?, ?)


2016-12-18 21:35:55,954 INFO sqlalchemy.engine.base.Engine ('obama', '2016-12-18 21:35:55.952563', 62008)


INFO:sqlalchemy.engine.base.Engine:('obama', '2016-12-18 21:35:55.952563', 62008)


2016-12-18 21:35:55,956 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


[2]

In [23]:
keyword3 = 'aleppo'
results3 = mc.sentenceCount(keyword3,
        solr_filter=[mc.publish_date_query( datetime.date(2016, 9, 1), datetime.date(2016, 10, 1) ),
                     'tags_id_media:1' ] )

In [24]:
insert_stmt3 = queries.insert().values(keywords=keyword3, count=results3['count'] )
str(insert_stmt3)

'INSERT INTO queries (keywords, timestamp, count) VALUES (:keywords, :timestamp, :count)'

In [25]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt3)
result.inserted_primary_key # print out the primary key it was assigned

2016-12-18 21:36:48,846 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp, count) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO queries (keywords, timestamp, count) VALUES (?, ?, ?)


2016-12-18 21:36:48,848 INFO sqlalchemy.engine.base.Engine ('aleppo', '2016-12-18 21:36:48.846518', 9526)


INFO:sqlalchemy.engine.base.Engine:('aleppo', '2016-12-18 21:36:48.846518', 9526)


2016-12-18 21:36:48,849 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


[3]

In [29]:
from sqlalchemy.sql import select

count = 0
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    count += row['count']
    
print "There were " + str(count) + " total sentences counted in queries in the QUERIES table"

2016-12-18 21:42:33,037 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp, queries.count 
FROM queries


INFO:sqlalchemy.engine.base.Engine:SELECT queries.id, queries.keywords, queries.timestamp, queries.count 
FROM queries


2016-12-18 21:42:33,039 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


There were 212202 total sentences counted in queries in the QUERIES table


### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [ ]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

In [ ]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

In [ ]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [ ]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [ ]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [ ]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [ ]:
query = Query(keywords="iguana")
query.keywords

In [ ]:
my_session.add(query)
my_session.commit()
query.id

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q